In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt
import inspect
import importlib

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import DataLoader, Data
from mpl_toolkits.mplot3d import Axes3D

from itertools import chain
from random import shuffle, sample

from torch.nn import Linear
import torch.nn.functional as F
from torch_scatter import scatter, segment_csr, scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_cluster import knn_graph, radius_graph
import trackml.dataset
import torch_geometric

sys.path.append('..')
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initial Exploration

In [7]:
from LightningModules.Processing.utils.detector_utils import load_detector, Detector_Thicknesses, Detector_Rotations, Detector_Pixel_Size

In [29]:
detector_path = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/detector.csv"

In [30]:
detector = pd.read_csv(detector_path)

In [8]:
thicknesses = Detector_Thicknesses(detector).get_thicknesses()
rotations = Detector_Rotations(detector).get_rotations()

  Extracting thicknesses...
  Done.
  Extracting rotations...
  Done.


In [31]:
detector_path = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml/detectors.csv"

In [32]:
detectors = pd.read_csv(detector_path)

In [15]:
thicknesses = Detector_Thicknesses(detector).get_thicknesses()
rotations = Detector_Rotations(detector).get_rotations()

  Extracting thicknesses...
  Done.
  Extracting rotations...
  Done.


In [34]:
detector = detector.rename(columns={"module_minhx": "module_minhu", "module_maxhx": "module_maxhu", "module_hy": "module_hv", "pitchX": "pitch_u", "pitchY": "pitch_v"})

In [35]:
detector

,volume_id,layer_id,module_id,cx,cy,cz,rot_xu,rot_xv,rot_xw,rot_yu,...,rot_yw,rot_zu,rot_zv,rot_zw,module_t,module_minhu,module_maxhu,module_hv,pitch_u,pitch_v
0,7,2,1,-65.7965,-5.17830,-1502.5,0.078459,-0.996917,0.0,-0.996917,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
1,7,2,2,-139.8510,-6.46568,-1502.0,0.046183,-0.998933,0.0,-0.998933,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
2,7,2,3,-138.6570,-19.34190,-1498.0,0.138156,-0.990410,0.0,-0.990410,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
3,7,2,4,-64.1764,-15.40740,-1498.0,0.233445,-0.972370,0.0,-0.972370,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
4,7,2,5,-136.2810,-32.05310,-1502.0,0.228951,-0.973438,0.0,-0.973438,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18723,18,12,94,-895.8950,291.09400,2952.5,0.309017,-0.951057,0.0,0.951057,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18724,18,12,95,-812.0900,161.53500,2947.5,0.195090,-0.980785,0.0,0.980785,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000
18725,18,12,96,-925.3150,176.51300,2955.5,0.187381,-0.982287,0.0,0.982287,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18726,18,12,97,-826.2270,54.15380,2944.5,0.065403,-0.997859,0.0,0.997859,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000


In [36]:
detectors

,volume_id,layer_id,module_id,cx,cy,cz,rot_xu,rot_xv,rot_xw,rot_yu,...,rot_yw,rot_zu,rot_zv,rot_zw,module_t,module_minhu,module_maxhu,module_hv,pitch_u,pitch_v
0,7,2,1,-65.7965,-5.17830,-1502.5,0.078459,-0.996917,0.0,-0.996917,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
1,7,2,2,-139.8510,-6.46568,-1502.0,0.046183,-0.998933,0.0,-0.998933,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
2,7,2,3,-138.6570,-19.34190,-1498.0,0.138156,-0.990410,0.0,-0.990410,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
3,7,2,4,-64.1764,-15.40740,-1498.0,0.233445,-0.972370,0.0,-0.972370,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
4,7,2,5,-136.2810,-32.05310,-1502.0,0.228951,-0.973438,0.0,-0.973438,...,0.0,0,0,-1,0.15,8.4,8.4,36,0.05,0.05625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18723,18,12,94,-895.8950,291.09400,2952.5,0.309017,-0.951057,0.0,0.951057,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18724,18,12,95,-812.0900,161.53500,2947.5,0.195090,-0.980785,0.0,0.980785,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000
18725,18,12,96,-925.3150,176.51300,2955.5,0.187381,-0.982287,0.0,0.982287,...,0.0,0,0,1,0.35,66.0,72.0,78,0.12,10.40000
18726,18,12,97,-826.2270,54.15380,2944.5,0.065403,-0.997859,0.0,0.997859,...,0.0,0,0,1,0.35,54.0,64.2,78,0.12,10.40000


In [38]:
(detectors == detector).all()

volume_id       True
layer_id        True
module_id       True
cx              True
cy              True
cz              True
rot_xu          True
rot_xv          True
rot_xw          True
rot_yu          True
rot_yv          True
rot_yw          True
rot_zu          True
rot_zv          True
rot_zw          True
module_t        True
module_minhu    True
module_maxhu    True
module_hv       True
pitch_u         True
pitch_v         True
dtype: bool

# Breakdown

## Top Level

In [3]:
from LightningModules.Processing.utils.detector_utils import load_detector, Detector_Thicknesses, Detector_Rotations, Detector_Pixel_Size
from LightningModules.Processing.utils.event_utils import build_event, get_cell_information

In [4]:
input_dir = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all"
detector_path = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml/detectors.csv"

In [5]:
%%time
cell_features = ['cell_count', 'cell_val', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi']
detector_orig, detector_proc = load_detector(detector_path)

Loading detector...
Detector loaded.
CPU times: user 46.3 ms, sys: 83.3 ms, total: 130 ms
Wall time: 130 ms


In [6]:
all_files = os.listdir(input_dir)
all_events = sorted(np.unique([os.path.join(input_dir, event[:14]) for event in all_files]))
event_file = all_events[0]

In [7]:
feature_scale = [1000, np.pi, 1000]
pt_min = 0
adjacent = False 
endcaps = True
layerless = True
layerwise = False
noise = False
cell_information = True

In [8]:
def select_hits(hits, truth, particles, pt_min=0, endcaps=False, noise=False):
    
    # Barrel volume and layer ids
    if endcaps:
        vlids = [(7, 2), (7, 4), (7, 6), (7, 8), (7, 10), (7, 12), (7, 14), (8, 2), (8, 4), (8, 6), (8, 8), (9, 2), (9, 4), (9, 6), (9, 8), (9, 10), (9, 12), (9, 14), (12, 2), (12, 4), (12, 6), (12, 8), (12, 10), (12, 12), (13, 2), (13, 4), (13, 6), (13, 8), (14, 2), (14, 4), (14, 6), (14, 8), (14, 10), (14, 12), (16, 2), (16, 4), (16, 6), (16, 8), (16, 10), (16, 12), (17, 2), (17, 4), (18, 2), (18, 4), (18, 6), (18, 8), (18, 10), (18, 12)]
    else:
        vlids = [(8,2), (8,4), (8,6), (8,8), (13,2), (13,4), (13,6), (13,8), (17,2), (17,4)]
    n_det_layers = len(vlids)
    # Select barrel layers and assign convenient layer number [0-9]
    vlid_groups = hits.groupby(['volume_id', 'layer_id'])
    hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])

    if noise is False:
        # Calculate particle transverse momentum
        pt = np.sqrt(particles.px**2 + particles.py**2)
        # Applies pt cut, removes noise hits
        particles = particles[pt > pt_min]
        truth = (truth[['hit_id', 'particle_id', 'tpx', 'tpy', 'weight']]
                 .merge(particles[['particle_id', 'vx', 'vy', 'vz']], on='particle_id'))
        truth = truth.assign(pt = np.sqrt(truth.tpx**2 + truth.tpy**2))
    else:
        # Calculate particle transverse momentum
        pt = np.sqrt(truth.tpx**2 + truth.tpy**2)
        # Applies pt cut
        truth = truth[pt > pt_min]
        truth.loc[truth['particle_id'] == 0,'particle_id'] = float('NaN')
        truth = truth.assign(pt = pt)

    # Calculate derived hits variables
    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    # Select the data columns we need
    hits = (hits[['hit_id', 'x', 'y', 'z', 'layer']]
            .assign(r=r, phi=phi)
            .merge(truth[['hit_id', 'particle_id', 'vx', 'vy', 'vz', 'pt', 'weight']], on='hit_id'))
    
    
    return hits

def build_event(event_file, pt_min, feature_scale, adjacent=True, endcaps=False, layerless=True, layerwise=True, noise=False):
    # Get true edge list using the ordering by R' = distance from production vertex of each particle
    
    hits, particles, truth = trackml.dataset.load_event(
        event_file, parts=['hits', 'particles', 'truth'])
    
    tic = tt()
    hits = select_hits(hits, truth, particles, pt_min=pt_min, endcaps=endcaps, noise=noise).assign(evtid=int(event_file[-9:]))
    layers = hits.layer.to_numpy()
    print(tt() - tic)
    return hits[['r', 'phi', 'z']].to_numpy() / feature_scale, hits.particle_id.to_numpy(), layers, hits['hit_id'].to_numpy(), hits.pt.to_numpy()

In [9]:
%%time
evtid = int(event_file[-9:])
X, pid, layers, hid, pt = build_event(event_file, pt_min, feature_scale, adjacent=adjacent, endcaps=endcaps, layerless=layerless, layerwise=layerwise, noise=noise)

0.09757351875305176
CPU times: user 405 ms, sys: 47.2 ms, total: 453 ms
Wall time: 455 ms


In [10]:
%%time
data = Data(
    x = torch.from_numpy(X).float(), 
    pid = torch.from_numpy(pid), 
    layers=torch.from_numpy(layers), 
    event_file=event_file, 
    hid = torch.from_numpy(hid),
    pt = torch.from_numpy(pt)
)

CPU times: user 974 µs, sys: 480 µs, total: 1.45 ms
Wall time: 799 µs


In [18]:
%%time
if cell_information:
    data = get_cell_information(data, cell_features, detector_orig, detector_proc, endcaps, noise)

CPU times: user 16.5 s, sys: 221 ms, total: 16.7 s
Wall time: 16.6 s


## Cell Calculation

In [11]:
from LightningModules.Processing.utils.cell_utils import get_one_event, augment_hit_features, get_cell_stats, extract_dir_new

### Setup

In [12]:
%%time
event_path = data.event_file
evtid = event_file[-4:]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs


In [13]:
%%time
hits_angles, cells = trackml.dataset.load_event(event_path, parts=['hits', 'cells'])

CPU times: user 248 ms, sys: 24.3 ms, total: 273 ms
Wall time: 318 ms


In [14]:
%%time
cell_stats = get_cell_stats(cells)
hits_angles['cell_count'] = cell_stats[:,0]
hits_angles['cell_val']   = cell_stats[:,1]

CPU times: user 25.5 ms, sys: 804 µs, total: 26.3 ms
Wall time: 25.5 ms


### Cell Direction

In [15]:
from LightningModules.Processing.utils.cell_utils import get_all_local_angles, get_all_rotated, cartesion_to_spherical, theta_to_eta

In [16]:
detector = detector_proc

In [17]:
%%time
l_u, l_v, l_w = get_all_local_angles(hits_angles, cells, detector)
g_matrix_all = get_all_rotated(hits_angles, detector, l_u, l_v, l_w)
hit_ids, cell_counts, cell_vals = hits_angles['hit_id'].to_numpy(), hits_angles['cell_count'].to_numpy(), hits_angles['cell_val'].to_numpy()

CPU times: user 56.4 ms, sys: 220 µs, total: 56.6 ms
Wall time: 55.8 ms


In [18]:
l_u, l_v = l_u.to_numpy(), l_v.to_numpy()

In [19]:
%%time
_, g_theta, g_phi = np.vstack(cartesion_to_spherical(*list(g_matrix_all.T)))
_, l_theta, l_phi = cartesion_to_spherical(l_u, l_v, l_w)
l_eta = theta_to_eta(l_theta)
g_eta = theta_to_eta(g_theta)
angles = np.vstack([hit_ids, cell_counts, cell_vals, l_eta, l_phi, l_u, l_v, l_w, g_eta, g_phi]).T

CPU times: user 27.6 ms, sys: 0 ns, total: 27.6 ms
Wall time: 26.8 ms


In [20]:
%%time
df_angles = pd.DataFrame(angles, columns=['cell_count', 'cell_val', 'hit_id', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi'])

CPU times: user 311 µs, sys: 0 ns, total: 311 µs
Wall time: 315 µs


## Post

In [21]:
%%time
hid = pd.DataFrame(data.hid.numpy(), columns = ["hit_id"])

CPU times: user 263 µs, sys: 153 µs, total: 416 µs
Wall time: 420 µs


In [ ]:
%%time
cell_data = hid.merge(df_angles, on="hit_id")[cell_features]

In [ ]:
cell_data = torch.from_numpy((hid.merge(df_angles, on="hit_id")[cell_features]).to_numpy()).float()
data.cell_data = cell_data

# GPU Development

In [3]:
import cupy as cp
import cudf
from torch.utils.dlpack import to_dlpack
from torch.utils.dlpack import from_dlpack

## Top Level

In [58]:
from LightningModules.Processing.utils.detector_utils import load_detector, Detector_Thicknesses, Detector_Rotations, Detector_Pixel_Size
from LightningModules.Processing.utils.event_utils import build_event, get_cell_information

In [59]:
input_dir = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all"
detector_path = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml/detectors.csv"

In [60]:
%%time
cell_features = ['cell_count', 'cell_val', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi']
detector_orig, detector_proc = load_detector(detector_path)

Loading detector...
Detector loaded.
CPU times: user 51 ms, sys: 116 ms, total: 167 ms
Wall time: 417 ms


In [12]:
all_files = os.listdir(input_dir)
all_events = sorted(np.unique([os.path.join(input_dir, event[:14]) for event in all_files]))
event_file = all_events[0]

In [13]:
feature_scale = [1000, np.pi, 1000]
pt_min = 0
adjacent = False 
endcaps = True
layerless = True
layerwise = False
noise = False
cell_information = True

In [300]:
def select_hits(hits, truth, particles, pt_min=0, endcaps=False, noise=False):
    
    # Barrel volume and layer ids
    if endcaps:
        vlids = [(7, 2), (7, 4), (7, 6), (7, 8), (7, 10), (7, 12), (7, 14), (8, 2), (8, 4), (8, 6), (8, 8), (9, 2), (9, 4), (9, 6), (9, 8), (9, 10), (9, 12), (9, 14), (12, 2), (12, 4), (12, 6), (12, 8), (12, 10), (12, 12), (13, 2), (13, 4), (13, 6), (13, 8), (14, 2), (14, 4), (14, 6), (14, 8), (14, 10), (14, 12), (16, 2), (16, 4), (16, 6), (16, 8), (16, 10), (16, 12), (17, 2), (17, 4), (18, 2), (18, 4), (18, 6), (18, 8), (18, 10), (18, 12)]
    else:
        vlids = [(8,2), (8,4), (8,6), (8,8), (13,2), (13,4), (13,6), (13,8), (17,2), (17,4)]
    n_det_layers = len(vlids)
    # Select barrel layers and assign convenient layer number [0-9]
    vlid_groups = hits.groupby(['volume_id', 'layer_id'])
    hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer=i)
                      for i in range(n_det_layers)])

    if noise is False:
        # Calculate particle transverse momentum
        pt = np.sqrt(particles.px**2 + particles.py**2)
        # Applies pt cut, removes noise hits
        particles = particles[pt > pt_min]
        truth = (truth[['hit_id', 'particle_id', 'tpx', 'tpy', 'weight']]
                 .merge(particles[['particle_id', 'vx', 'vy', 'vz']], on='particle_id'))
        truth = truth.assign(pt = np.sqrt(truth.tpx**2 + truth.tpy**2))
    else:
        # Calculate particle transverse momentum
        pt = np.sqrt(truth.tpx**2 + truth.tpy**2)
        # Applies pt cut
        truth = truth[pt > pt_min]
        truth.loc[truth['particle_id'] == 0,'particle_id'] = float('NaN')
        truth = truth.assign(pt = pt)

    # Calculate derived hits variables
    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    # Select the data columns we need
    hits = (hits[['hit_id', 'x', 'y', 'z', 'layer']]
            .assign(r=r, phi=phi)
            .merge(truth[['hit_id', 'particle_id', 'vx', 'vy', 'vz', 'pt', 'weight']], on='hit_id'))
    
    
    return hits

def build_event(event_file, pt_min, feature_scale, adjacent=True, endcaps=False, layerless=True, layerwise=True, noise=False):
    # Get true edge list using the ordering by R' = distance from production vertex of each particle
    
    hits, particles, truth = trackml.dataset.load_event(
        event_file, parts=['hits', 'particles', 'truth'])
    
    tic = tt()
    hits = select_hits(hits, truth, particles, pt_min=pt_min, endcaps=endcaps, noise=noise).assign(evtid=int(event_file[-9:]))
    layers = hits.layer.to_numpy()
#     print(tt() - tic)
    return hits[['r', 'phi', 'z']].to_numpy() / feature_scale, hits.particle_id.to_numpy(), layers, hits['hit_id'].to_numpy(), hits.pt.to_numpy()

In [41]:
def select_hits_gpu(hits, truth, particles, pt_min=0, endcaps=False, noise=False):
    
    if endcaps:
        vlids = [(7, 2), (7, 4), (7, 6), (7, 8), (7, 10), (7, 12), (7, 14), (8, 2), (8, 4), (8, 6), (8, 8), (9, 2), (9, 4), (9, 6), (9, 8), (9, 10), (9, 12), (9, 14), (12, 2), (12, 4), (12, 6), (12, 8), (12, 10), (12, 12), (13, 2), (13, 4), (13, 6), (13, 8), (14, 2), (14, 4), (14, 6), (14, 8), (14, 10), (14, 12), (16, 2), (16, 4), (16, 6), (16, 8), (16, 10), (16, 12), (17, 2), (17, 4), (18, 2), (18, 4), (18, 6), (18, 8), (18, 10), (18, 12)]
    else:
        vlids = [(8,2), (8,4), (8,6), (8,8), (13,2), (13,4), (13,6), (13,8), (17,2), (17,4)]
    layernum_lookup = np.empty((19, 15))
    for i, entry in enumerate(vlids):
        layernum_lookup[entry[0], entry[1]] = i

    hits = hits.apply_rows(vol_to_layernum,
                   incols=['volume_id', 'layer_id'],
                    outcols=dict(layer=np.int64),
                   kwargs = {})

    if noise is False:
        pt = cp.sqrt(particles.px**2 + particles.py**2)
        particles =  particles[pt > pt_min]
        truth = (truth[['hit_id', 'particle_id', 'tpx', 'tpy', 'weight']]).merge(particles[['particle_id', 'vx', 'vy', 'vz']], on='particle_id')
        pt = cp.sqrt(truth.tpx**2 + truth.tpy**2)
        truth = truth.assign(pt = pt)
    else:
        print("TO IMPLEMENT GPU NOISY")

    # Calculate derived hits variables
    r = cp.sqrt(hits.x**2 + hits.y**2)
    phi = cp.arctan2(hits.y, hits.x)
    # Select the data columns we need
    hits = (hits[['hit_id', 'x', 'y', 'z', 'layer']]
            .assign(r=r, phi=phi)
            .merge(truth[['hit_id', 'particle_id', 'vx', 'vy', 'vz', 'pt', 'weight']], on='hit_id'))
    
    return hits

def build_event_gpu(event_file, pt_min, feature_scale, adjacent=True, endcaps=False, layerless=True, layerwise=True, noise=False):
    # Get true edge list using the ordering by R' = distance from production vertex of each particle
    
    hits = cudf.io.read_csv("{}-hits.csv".format(event_file))
    particles = cudf.io.read_csv("{}-particles.csv".format(event_file))
    truth = cudf.io.read_csv("{}-truth.csv".format(event_file))
    cells = cudf.io.read_csv("{}-cells.csv".format(event_file))
    
    tic = tt()
    hits = select_hits_gpu(hits, truth, particles, pt_min=pt_min, endcaps=endcaps, noise=noise).assign(evtid=int(event_file[-9:]))
    layers = hits.layer
#     print(tt() - tic)
#     return hits[['r', 'phi', 'z']].as_matrix() / feature_scale, hits.particle_id.values_host, layers, hits['hit_id'].values_host, hits.pt.values_host
    return hits[['r', 'phi', 'z']] / feature_scale, hits.particle_id, layers, hits['hit_id'], hits.pt

In [42]:
event_file = "/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/event000021000"

In [227]:
def vol_to_layernum(volume_id, layer_id, layer):
    for i, (in1, in2) in enumerate(zip(volume_id, layer_id)):
        layer[i] = layernum_lookup[in1, in2]

In [301]:
n_iters = 100
evtid = int(event_file[-9:])
tic = tt()

for i in range(n_iters):
    X, pid, layers, hid, pt = build_event(event_file, pt_min, feature_scale, adjacent=adjacent, endcaps=endcaps, layerless=layerless, layerwise=layerwise, noise=noise)
print((tt() - tic) / n_iters)

0.33290465354919435


In [304]:
n_iters = 100
evtid = int(event_file[-9:])
tic = tt()

for i in range(n_iters):
    X, pid, layers, hid, pt = build_event_gpu(event_file, pt_min, feature_scale, adjacent=adjacent, endcaps=endcaps, layerless=layerless, layerwise=layerwise, noise=noise)

print((tt() - tic) / n_iters)

0.12087683200836181


In [327]:
%%time
X_np = X.as_matrix()
X_torch = torch.from_numpy(X_np).to("cuda")

CPU times: user 0 ns, sys: 3.79 ms, total: 3.79 ms
Wall time: 3.33 ms


In [328]:
%%time
X_dlpack = X.to_dlpack()
X_torch = from_dlpack(X_dlpack)

CPU times: user 776 µs, sys: 0 ns, total: 776 µs
Wall time: 620 µs


X is not so time consuming: What if the matrix was MUCH larger...

In [35]:
n_edges = 1000000
edge_test = np.random.rand(2, n_edges)

In [36]:
edges_cp = cp.array(edge_test)

In [38]:
%%time
edges_np = cp.asnumpy(edges_cp)
edges_torch = torch.from_numpy(edges_np)
edges_torch = edges_torch.to("cuda")

CPU times: user 11 ms, sys: 7.12 ms, total: 18.1 ms
Wall time: 17.1 ms


In [39]:
%%time
edges_dlpack = edges_cp.toDlpack() # Careful, CuDF and CuPy capitalise this method differently!
edges_torch = from_dlpack(edges_dlpack)

CPU times: user 37 µs, sys: 30 µs, total: 67 µs
Wall time: 69.4 µs


In [10]:
%%time
data = Data(
    x = torch.from_numpy(X).float(), 
    pid = torch.from_numpy(pid), 
    layers=torch.from_numpy(layers), 
    event_file=event_file, 
    hid = torch.from_numpy(hid),
    pt = torch.from_numpy(pt)
)

CPU times: user 974 µs, sys: 480 µs, total: 1.45 ms
Wall time: 799 µs


In [18]:
%%time
if cell_information:
    data = get_cell_information(data, cell_features, detector_orig, detector_proc, endcaps, noise)

CPU times: user 16.5 s, sys: 221 ms, total: 16.7 s
Wall time: 16.6 s


## Cell Calculation

In [61]:
from LightningModules.Processing.utils.cell_utils import get_one_event, augment_hit_features, get_cell_stats, extract_dir_new

### Setup

In [62]:
%%time
hits = cudf.io.read_csv("{}-hits.csv".format(event_file))
cells = cudf.io.read_csv("{}-cells.csv".format(event_file))

CPU times: user 13.3 ms, sys: 9.16 ms, total: 22.4 ms
Wall time: 23.3 ms


In [63]:
%%time
cell_stats = get_cell_stats(cells)
hits['cell_count'] = cell_stats[:,0]
hits['cell_val']   = cell_stats[:,1]

CPU times: user 17 ms, sys: 8.86 ms, total: 25.9 ms
Wall time: 25.3 ms


### Cell Direction

In [64]:
from LightningModules.Processing.utils.cell_utils import get_all_local_angles, get_all_rotated, cartesion_to_spherical, theta_to_eta

In [65]:
detector = detector_proc

In [68]:
%%time
l_u, l_v, l_w = get_all_local_angles(hits, cells, detector)
g_matrix_all = get_all_rotated(hits, detector, l_u, l_v, l_w)
# hit_ids, cell_counts, cell_vals = hits['hit_id'].to_numpy(), hits['cell_count'].to_numpy(), hits['cell_val'].to_numpy()

ValueError: object __array__ method not producing an array

In [18]:
l_u, l_v = l_u.to_numpy(), l_v.to_numpy()

In [19]:
%%time
_, g_theta, g_phi = np.vstack(cartesion_to_spherical(*list(g_matrix_all.T)))
_, l_theta, l_phi = cartesion_to_spherical(l_u, l_v, l_w)
l_eta = theta_to_eta(l_theta)
g_eta = theta_to_eta(g_theta)
angles = np.vstack([hit_ids, cell_counts, cell_vals, l_eta, l_phi, l_u, l_v, l_w, g_eta, g_phi]).T

CPU times: user 27.6 ms, sys: 0 ns, total: 27.6 ms
Wall time: 26.8 ms


In [20]:
%%time
df_angles = pd.DataFrame(angles, columns=['cell_count', 'cell_val', 'hit_id', 'leta', 'lphi', 'lx', 'ly', 'lz', 'geta', 'gphi'])

CPU times: user 311 µs, sys: 0 ns, total: 311 µs
Wall time: 315 µs


## Post

In [21]:
%%time
hid = pd.DataFrame(data.hid.numpy(), columns = ["hit_id"])

CPU times: user 263 µs, sys: 153 µs, total: 416 µs
Wall time: 420 µs


In [ ]:
%%time
cell_data = hid.merge(df_angles, on="hit_id")[cell_features]

In [ ]:
cell_data = torch.from_numpy((hid.merge(df_angles, on="hit_id")[cell_features]).to_numpy()).float()
data.cell_data = cell_data